## Cricket match Analysis

### DATA EXTRACTION AND PROCESSING

### * Extracting the data from the json files and converting it into a data frame 

In [2]:
import os
import pandas as pd
import json
import mysql.connector


## IPL Data Frame

In [2]:

class CricketMatchData:
    def __init__(self, json_data):
        self.data = json_data

    def create_dataframe(self):
        rows = []

        # Extract common information from the info section
        info = self.data['info']
        meta = self.data['meta']
        common_data = {
            'match_date': info['dates'][0],
            'city': info.get('city', None),
            'venue': info.get('venue', None),
            'match_type': info['event']['name'] + ' ' + info['match_type'],
            'match_number': info['event'].get('match_number', None),
            'winner': info['outcome'].get('winner', None),
            'player_of_match': info['player_of_match'][0] if 'player_of_match' in info else None,
            'team1': info['teams'][0],
            'team2': info['teams'][1]
        }

        # Iterate through innings and deliveries
        for inning in self.data['innings']:
            team = inning['team']

            for over in inning['overs']:
                over_number = over['over']

                for delivery in over['deliveries']:
                    row = {
                        **common_data,  # Add common data using unpacking operator ** 
                        'batting_team': team,
                        'overs': over_number,
                        'batter': delivery['batter'],
                        'bowler': delivery['bowler'],
                        'non_striker': delivery['non_striker'],
                        'runs_batter': delivery['runs']['batter'],
                        'runs_extras': delivery['runs']['extras'],
                        'runs_total': delivery['runs']['total'],
                        'wicket_type': None,
                        'player_out': None,
                        'fielder': None
                    }

                    # Check for wickets
                    if 'wickets' in delivery:
                        row['wicket_type'] = delivery['wickets'][0]['kind']
                        row['player_out'] = delivery['wickets'][0]['player_out']
                        if 'fielders' in delivery['wickets'][0]:
                            row['fielder'] = delivery['wickets'][0]['fielders'][0]['name']

                    rows.append(row)

        # Create DataFrame
        return pd.DataFrame(rows)


def process_cricket_data(folder_path):
    all_dataframes = []

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a JSON file
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            try:
                # Read the JSON file
                with open(file_path, 'r') as json_file:
                    json_data = json.load(json_file)

                # Process the data using CricketMatchData class
                cricket_data = CricketMatchData(json_data)
                df = cricket_data.create_dataframe()

                # Append the resulting DataFrame
                all_dataframes.append(df)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    # Combine all DataFrames into one
    if all_dataframes:
        final_dataframe = pd.concat(all_dataframes, ignore_index=True)
        return final_dataframe
    else:
        print("No valid dataframes were created.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was processed

# Example usage
folder_path = "/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Data/ipl_json"  # Replace with your folder path
ipl_df = process_cricket_data(folder_path)
ipl_df 



,match_date,city,venue,match_type,match_number,winner,player_of_match,team1,team2,batting_team,overs,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,wicket_type,player_out,fielder
0,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,None,None,None
1,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,None,None,None
2,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,4,0,4,None,None,None
3,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,None,None,None
4,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262143,2018-05-05,Hyderabad,Rajiv Gandhi International Stadium,Indian Premier League T20,36,Sunrisers Hyderabad,Rashid Khan,Delhi Daredevils,Sunrisers Hyderabad,Sunrisers Hyderabad,19,YK Pathan,DT Christian,KS Williamson,2,0,2,None,None,None
262144,2018-05-05,Hyderabad,Rajiv Gandhi International Stadium,Indian Premier League T20,36,Sunrisers Hyderabad,Rashid Khan,Delhi Daredevils,Sunrisers Hyderabad,Sunrisers Hyderabad,19,YK Pathan,DT Christian,KS Williamson,6,0,6,None,None,None
262145,2018-05-05,Hyderabad,Rajiv Gandhi International Stadium,Indian Premier League T20,36,Sunrisers Hyderabad,Rashid Khan,Delhi Daredevils,Sunrisers Hyderabad,Sunrisers Hyderabad,19,YK Pathan,DT Christian,KS Williamson,4,0,4,None,None,None
262146,2018-05-05,Hyderabad,Rajiv Gandhi International Stadium,Indian Premier League T20,36,Sunrisers Hyderabad,Rashid Khan,Delhi Daredevils,Sunrisers Hyderabad,Sunrisers Hyderabad,19,YK Pathan,DT Christian,KS Williamson,1,0,1,None,None,None


### ODIs Data Frame 

In [3]:
import os
import pandas as pd
import json

class CricketMatchData:
    def __init__(self, json_data):
        self.data = json_data

    def create_dataframe(self):
        rows = []

        # Extract common information from the info section
        info = self.data['info']
        meta = self.data['meta']
        common_data = {
            'match_date': info['dates'][0],
            'city': info.get('city', None),
            'venue': info.get('venue', None),
            'match_type': info.get('match_type', None),
            'match_number': info.get('match_type_number', None),
            'winner': info.get('outcome', {}).get('winner', None),
            'player_of_match': info.get('player_of_match', [None])[0],
            'team1': info['teams'][0],
            'team2': info['teams'][1]
        }

        # Include event details if present
        if 'event' in info:
            common_data['event_name'] = info['event'].get('name', None)
            common_data['event_match_number'] = info['event'].get('match_number', None)

        # Iterate through innings and deliveries
        for inning in self.data['innings']:
            team = inning['team']

            for over in inning['overs']:
                over_number = over['over']

                for delivery in over['deliveries']:
                    row = {
                        **common_data,  # Add common data using unpacking operator ** 
                        'batting_team': team,
                        'overs': over_number,
                        'batter': delivery['batter'],
                        'bowler': delivery['bowler'],
                        'non_striker': delivery['non_striker'],
                        'runs_batter': delivery['runs']['batter'],
                        'runs_extras': delivery['runs']['extras'],
                        'runs_total': delivery['runs']['total'],
                        'wicket_type': None,
                        'player_out': None,
                        'fielder': None
                    }

                    # Check for wickets
                    if 'wickets' in delivery:
                        row['wicket_type'] = delivery['wickets'][0]['kind']
                        row['player_out'] = delivery['wickets'][0]['player_out']
                        if 'fielders' in delivery['wickets'][0]:
                            row['fielder'] = delivery['wickets'][0]['fielders'][0]['name']

                    rows.append(row)

        # Create DataFrame
        return pd.DataFrame(rows)


def process_cricket_data(folder_path):
    all_dataframes = []

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a JSON file
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            try:
                # Read the JSON file
                with open(file_path, 'r') as json_file:
                    json_data = json.load(json_file)

                # Process the data using CricketMatchData class
                cricket_data = CricketMatchData(json_data)
                df = cricket_data.create_dataframe()

                # Append the resulting DataFrame
                all_dataframes.append(df)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    # Combine all DataFrames into one
    if all_dataframes:
        final_dataframe = pd.concat(all_dataframes, ignore_index=True)
        return final_dataframe
    else:
        print("No valid dataframes were created.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was processed

# Example usage
folder_path = "/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Data/odis_json"  # Replace with your folder path
odis_df = process_cricket_data(folder_path)

odis_df

/var/folders/q9/1vgcrp_s61j9_p6vmrcdf1h00000gn/T/ipykernel_3296/3318431575.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat(all_dataframes, ignore_index=True)


,match_date,city,venue,match_type,match_number,winner,player_of_match,team1,team2,event_name,...,overs,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,wicket_type,player_out,fielder
0,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,4,None,None,None
1,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0,lbw,RD Gaikwad,None
2,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,None,None,None
3,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,None,None,None
4,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546107,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None
1546108,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None
1546109,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None
1546110,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None


## T20's Data Frame

In [4]:
import os
import pandas as pd
import json

class CricketMatchData:
    def __init__(self, json_data):
        self.data = json_data

    def create_dataframe(self):
        rows = []

        # Extract common information from the info section
        info = self.data['info']
        meta = self.data['meta']
        common_data = {
            'match_date': info['dates'][0],
            'city': info.get('city', None),
            'venue': info.get('venue', None),
            'match_type': info.get('match_type', None),
            'match_number': info.get('match_type_number', None),
            'winner': info.get('outcome', {}).get('winner', None),
            'player_of_match': info.get('player_of_match', [None])[0],
            'team1': info['teams'][0],
            'team2': info['teams'][1]
        }

        # Include event details if present
        if 'event' in info:
            common_data['event_name'] = info['event'].get('name', None)
            common_data['event_match_number'] = info['event'].get('match_number', None)

        # Iterate through innings and deliveries
        for inning in self.data['innings']:
            team = inning['team']

            for over in inning['overs']:
                over_number = over['over']

                for delivery in over['deliveries']:
                    row = {
                        **common_data,  # Add common data using unpacking operator ** 
                        'batting_team': team,
                        'overs': over_number,
                        'batter': delivery['batter'],
                        'bowler': delivery['bowler'],
                        'non_striker': delivery['non_striker'],
                        'runs_batter': delivery['runs']['batter'],
                        'runs_extras': delivery['runs']['extras'],
                        'runs_total': delivery['runs']['total'],
                        'wicket_type': None,
                        'player_out': None,
                        'fielder': None
                    }

                    # Check for wickets
                    if 'wickets' in delivery:
                        row['wicket_type'] = delivery['wickets'][0]['kind']
                        row['player_out'] = delivery['wickets'][0]['player_out']
                        if 'fielders' in delivery['wickets'][0]:
                            if "name" in delivery['wickets'][0]['fielders'][0]:
                                row['fielder'] = delivery['wickets'][0]['fielders'][0]['name']
                            elif 'substitute' in delivery['wickets'][0]['fielders'][0]:
                                row['fielder'] = None
                                
                    rows.append(row)

        # Create DataFrame
        return pd.DataFrame(rows)


def process_cricket_data(folder_path):
    all_dataframes = []

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a JSON file
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            try:
                # Read the JSON file
                with open(file_path, 'r') as json_file:
                    json_data = json.load(json_file)

                # Process the data using CricketMatchData class
                cricket_data = CricketMatchData(json_data)
                df = cricket_data.create_dataframe()

                # Append the resulting DataFrame
                all_dataframes.append(df)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    # Combine all DataFrames into one
    if all_dataframes:
        final_dataframe = pd.concat(all_dataframes, ignore_index=True)
        return final_dataframe
    else:
        print("No valid dataframes were created.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was processed

# Example usage
folder_path = "/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Data/t20s_json"  # Data folder path
t20s_df = process_cricket_data(folder_path)

t20s_df

/var/folders/q9/1vgcrp_s61j9_p6vmrcdf1h00000gn/T/ipykernel_3296/1931109095.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat(all_dataframes, ignore_index=True)


,match_date,city,venue,match_type,match_number,winner,player_of_match,team1,team2,event_name,...,overs,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,wicket_type,player_out,fielder
0,2024-07-30,Kandy,Pallekele International Cricket Stadium,T20,2769,None,Washington Sundar,India,Sri Lanka,India tour of Sri Lanka,...,0,YBK Jaiswal,C Wickramasinghe,Shubman Gill,0,0,0,None,None,None
1,2024-07-30,Kandy,Pallekele International Cricket Stadium,T20,2769,None,Washington Sundar,India,Sri Lanka,India tour of Sri Lanka,...,0,YBK Jaiswal,C Wickramasinghe,Shubman Gill,1,0,1,None,None,None
2,2024-07-30,Kandy,Pallekele International Cricket Stadium,T20,2769,None,Washington Sundar,India,Sri Lanka,India tour of Sri Lanka,...,0,Shubman Gill,C Wickramasinghe,YBK Jaiswal,0,0,0,None,None,None
3,2024-07-30,Kandy,Pallekele International Cricket Stadium,T20,2769,None,Washington Sundar,India,Sri Lanka,India tour of Sri Lanka,...,0,Shubman Gill,C Wickramasinghe,YBK Jaiswal,1,0,1,None,None,None
4,2024-07-30,Kandy,Pallekele International Cricket Stadium,T20,2769,None,Washington Sundar,India,Sri Lanka,India tour of Sri Lanka,...,0,YBK Jaiswal,C Wickramasinghe,Shubman Gill,1,0,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917206,2019-11-05,Nelson,Saxton Oval,T20,1001,New Zealand,C de Grandhomme,New Zealand,England,England tour of New Zealand,...,19,TK Curran,TG Southee,S Mahmood,0,0,0,None,None,None
917207,2019-11-05,Nelson,Saxton Oval,T20,1001,New Zealand,C de Grandhomme,New Zealand,England,England tour of New Zealand,...,19,TK Curran,TG Southee,S Mahmood,1,0,1,None,None,None
917208,2019-11-05,Nelson,Saxton Oval,T20,1001,New Zealand,C de Grandhomme,New Zealand,England,England tour of New Zealand,...,19,S Mahmood,TG Southee,TK Curran,0,0,0,None,None,None
917209,2019-11-05,Nelson,Saxton Oval,T20,1001,New Zealand,C de Grandhomme,New Zealand,England,England tour of New Zealand,...,19,S Mahmood,TG Southee,TK Curran,1,0,1,None,None,None


## Test Data Frame

In [5]:
import os
import pandas as pd
import json

class CricketMatchData:
    def __init__(self, json_data):
        self.data = json_data

    def create_dataframe(self):
        rows = []

        # Extract common information from the info section
        info = self.data['info']
        meta = self.data['meta']
        common_data = {
            'match_start_date': info['dates'][0],
            'match_end_date': info['dates'][-1],
            'city': info.get('city', None),
            'venue': info.get('venue', None),
            'match_type': info.get('match_type', None),
            'match_number': info.get('match_type_number', None),
            'winner': info.get('outcome', {}).get('winner', None),
            'player_of_match': info.get('player_of_match', [None])[0],
            'team1': info['teams'][0],
            'team2': info['teams'][1]
        }

        # Include event details if present
        if 'event' in info:
            common_data['event_name'] = info['event'].get('name', None)
            common_data['event_match_number'] = info['event'].get('match_number', None)

        # Iterate through innings and deliveries
        for inning in self.data['innings']:
            team = inning['team']

            for over in inning['overs']:
                over_number = over['over']

                for delivery in over['deliveries']:
                    row = {
                        **common_data,  # Add common data using unpacking operator ** 
                        'batting_team': team,
                        'overs': over_number,
                        'batter': delivery['batter'],
                        'bowler': delivery['bowler'],
                        'non_striker': delivery['non_striker'],
                        'runs_batter': delivery['runs']['batter'],
                        'runs_extras': delivery['runs']['extras'],
                        'runs_total': delivery['runs']['total'],
                        'wicket_type': None,
                        'player_out': None,
                        'fielder': None
                    }

                    # Check for wickets
                    if 'wickets' in delivery:
                        row['wicket_type'] = delivery['wickets'][0]['kind']
                        row['player_out'] = delivery['wickets'][0]['player_out']
                        if 'fielders' in delivery['wickets'][0]:
                            if "name" in delivery['wickets'][0]['fielders'][0]:
                                row['fielder'] = delivery['wickets'][0]['fielders'][0]['name']
                            elif 'substitute' in delivery['wickets'][0]['fielders'][0]:
                                row['fielder'] = None
                                
                    rows.append(row)

        # Create DataFrame
        return pd.DataFrame(rows)


def process_cricket_data(folder_path):
    all_dataframes = []

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a JSON file
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            try:
                # Read the JSON file
                with open(file_path, 'r') as json_file:
                    json_data = json.load(json_file)

                # Process the data using CricketMatchData class
                cricket_data = CricketMatchData(json_data)
                df = cricket_data.create_dataframe()

                # Append the resulting DataFrame
                all_dataframes.append(df)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    # Combine all DataFrames into one
    if all_dataframes:
        final_dataframe = pd.concat(all_dataframes, ignore_index=True)
        return final_dataframe
    else:
        print("No valid dataframes were created.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was processed

# Example usage
folder_path = "/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Data/tests_json"  # Data folder path
test_df = process_cricket_data(folder_path)

test_df

/var/folders/q9/1vgcrp_s61j9_p6vmrcdf1h00000gn/T/ipykernel_3296/654683784.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat(all_dataframes, ignore_index=True)


,match_start_date,match_end_date,city,venue,match_type,match_number,winner,player_of_match,team1,team2,...,overs,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,wicket_type,player_out,fielder
0,2020-12-17,2020-12-19,Adelaide,Adelaide Oval,Test,2396,Australia,TD Paine,India,Australia,...,0,PP Shaw,MA Starc,MA Agarwal,0,0,0,None,None,None
1,2020-12-17,2020-12-19,Adelaide,Adelaide Oval,Test,2396,Australia,TD Paine,India,Australia,...,0,PP Shaw,MA Starc,MA Agarwal,0,0,0,bowled,PP Shaw,None
2,2020-12-17,2020-12-19,Adelaide,Adelaide Oval,Test,2396,Australia,TD Paine,India,Australia,...,0,CA Pujara,MA Starc,MA Agarwal,0,0,0,None,None,None
3,2020-12-17,2020-12-19,Adelaide,Adelaide Oval,Test,2396,Australia,TD Paine,India,Australia,...,0,CA Pujara,MA Starc,MA Agarwal,1,0,1,None,None,None
4,2020-12-17,2020-12-19,Adelaide,Adelaide Oval,Test,2396,Australia,TD Paine,India,Australia,...,0,MA Agarwal,MA Starc,CA Pujara,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1669264,2017-10-21,2017-10-25,Bulawayo,Queens Sports Club,Test,2279,West Indies,D Bishoo,West Indies,Zimbabwe,...,89,CB Mpofu,D Bishoo,KM Jarvis,0,0,0,None,None,None
1669265,2017-10-21,2017-10-25,Bulawayo,Queens Sports Club,Test,2279,West Indies,D Bishoo,West Indies,Zimbabwe,...,90,KM Jarvis,RL Chase,CB Mpofu,1,0,1,None,None,None
1669266,2017-10-21,2017-10-25,Bulawayo,Queens Sports Club,Test,2279,West Indies,D Bishoo,West Indies,Zimbabwe,...,90,CB Mpofu,RL Chase,KM Jarvis,4,0,4,None,None,None
1669267,2017-10-21,2017-10-25,Bulawayo,Queens Sports Club,Test,2279,West Indies,D Bishoo,West Indies,Zimbabwe,...,90,CB Mpofu,RL Chase,KM Jarvis,2,0,2,None,None,None


### store the dataframe in a csv file to a specific folder

In [27]:

odis_df.to_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/odis_df.csv", index=False)
odis_df

,match_date,city,venue,match_type,match_number,winner,player_of_match,team1,team2,event_name,...,overs,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,wicket_type,player_out,fielder
0,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,4,None,None,None
1,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0,lbw,RD Gaikwad,None
2,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,None,None,None
3,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,None,None,None
4,2023-12-19,Gqeberha,"St George's Park, Gqeberha",ODI,4714,South Africa,T de Zorzi,India,South Africa,India tour of South Africa,...,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546107,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None
1546108,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None
1546109,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None
1546110,2012-02-03,Dunedin,University Oval,ODI,3230,New Zealand,MJ Guptill,New Zealand,Zimbabwe,Zimbabwe in New Zealand ODI Series,...,40,RW Price,TG Southee,KM Jarvis,0,0,0,None,None,None


## Connect with SQL server 

In [7]:

class Database:
    def __init__(self, host, user, password):
        self.host = host
        self.user = user
        self.password = password

    def connect(self):
        connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password
        )
        cursor = connection.cursor()
        return connection , cursor
    
    def create_table(self, table_name, columns):
        query = "CREATE TABLE IF NOT EXISTS " + table_name + " (" + ", ".join(columns) + ")"
        cursor.execute(query)
        connection.commit()

class Data_updation:
    def __init__(self, connection, cursor):
        self.connection = connection
        self.cursor = cursor

    def insert_data(self, table_name, data_frame, chunk_size=5000):
        import numpy as np

        # Clean DataFrame: Replace real NaNs, string 'nan', and NaT with None
        data_frame = data_frame.replace(['nan', 'NaN', 'NaT'], None)
        data_frame = data_frame.where(pd.notnull(data_frame), None)

        # Convert every value to safe MySQL-friendly formats
        def clean_value(val):
            if isinstance(val, float) and np.isnan(val):
                return None
            elif isinstance(val, str) and val.strip().lower() == 'nan':
                return None
            return val

        columns = ", ".join(data_frame.columns)
        placeholders = ", ".join(["%s"] * len(data_frame.columns))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        for i in range(0, len(data_frame), chunk_size):
            chunk = data_frame.iloc[i:i+chunk_size]
            
            # Clean each row before insert
            data = [tuple(clean_value(val) for val in row) for _, row in chunk.iterrows()]

            try:
                self.cursor.executemany(query, data)
                self.connection.commit()
            except Exception as e:
                print(f"Error in chunk {i} to {i+chunk_size}: {e}")

        

db = Database('localhost', "root", "MuraliSQL@1234")
query = "create database if not exists Cricket_match_data_analysis"
connection , cursor = db.connect()

cursor.execute(query)
query ="use Cricket_match_data_analysis"
cursor.execute(query)
connection.commit()


In [11]:

# Create tables


table_1 =db.create_table('ipl_table',['match_date DATE', 'city VARCHAR(200)', 'venue VARCHAR(200)','match_type VARCHAR(200)', 'match_number INT',
                                'winner VARCHAR(200)', 'player_of_match VARCHAR(200)','team1 VARCHAR(200)', 'team2 VARCHAR(200)',
                                'batting_team VARCHAR(200)', 'overs INT','batter VARCHAR(200)', 'bowler VARCHAR(200)',
                                'non_striker VARCHAR(200)', 'runs_batter INT','runs_extras INT', 'runs_total INT',
                                'wicket_type VARCHAR(200)', 'player_out VARCHAR(200)','fielder VARCHAR(200)'])


table_2 =db.create_table('odi_table',['match_date DATE', 'city VARCHAR(200)', 'venue VARCHAR(200)', 'match_type VARCHAR(200)', 'match_number INT',
                                     'winner VARCHAR(200)', 'player_of_match VARCHAR(200)', 'team1 VARCHAR(200)', 'team2 VARCHAR(200)', 
                                     'event_name VARCHAR(200)', 'event_match_number INT', 'batting_team VARCHAR(200)', 'overs INT', 'batter VARCHAR(200)', 'bowler VARCHAR(200)', 
                                     'non_striker VARCHAR(200)', 'runs_batter INT', 'runs_extras INT', 'runs_total INT',
                                     'wicket_type VARCHAR(200)', 'player_out VARCHAR(200)', 'fielder VARCHAR(200)'])
                                     


table_3 =db.create_table('t20s_table',['match_date DATE', 'city VARCHAR(200)', 'venue VARCHAR(200)', 'match_type VARCHAR(200)', 'match_number INT',
                                     'winner VARCHAR(200)', 'player_of_match VARCHAR(200)', 'team1 VARCHAR(200)', 'team2 VARCHAR(200)',
                                     'event_name VARCHAR(200)', 'event_match_number INT', 'batting_team VARCHAR(200)', 'overs INT',
                                     'batter VARCHAR(200)', 'bowler VARCHAR(200)', 'non_striker VARCHAR(200)', 'runs_batter INT',
                                     'runs_extras INT', 'runs_total INT', 'wicket_type VARCHAR(200)', 'player_out VARCHAR(200)','fielder VARCHAR(200)'])
                                     


table_4 =db.create_table('test_table',['match_start_date DATE', 'match_end_date DATE', 'city VARCHAR(200)', 'venue VARCHAR(200)', 'match_type VARCHAR(200)', 'match_number INT',
                                       'winner VARCHAR(200)', 'player_of_match VARCHAR(200)', 'team1 VARCHAR(200)', 'team2 VARCHAR(200)',
                                       'event_name VARCHAR(200)', 'event_match_number INT', 'batting_team VARCHAR(200)', 'overs INT',
                                       'batter VARCHAR(200)', 'bowler VARCHAR(200)', 'non_striker VARCHAR(200)', 'runs_batter INT',
                                       'runs_extras INT', 'runs_total INT', 'wicket_type VARCHAR(200)', 'player_out VARCHAR(200)', 'fielder VARCHAR(200)'])


# insert the data frames in the database 



data_updater = Data_updation(connection, cursor)

#data_updater.insert_data('ipl_table', ipl_df, chunk_size=5000)
#data_updater.insert_data('odi_table', odis_df, chunk_size=5000)
#data_updater.insert_data('t20s_table', t20s_df, chunk_size=5000)
#data_updater.insert_data('test_table', test_df, chunk_size=5000)



print("Python code executed successfully and connected to the database and tables have been updated.!")

Python code executed successfully and connected to the database and tables have been updated.!


## Analysis with the Dataframes 

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


# set rows and columns to max display of 50 rows 
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)


### Loading the data 

In [4]:
ipl_df = pd.read_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/ipl_df.csv")
odis_df = pd.read_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/odis_df.csv")
t20s_df = pd.read_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/t20s_df.csv")
test_df = pd.read_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/test_df.csv")



/var/folders/q9/1vgcrp_s61j9_p6vmrcdf1h00000gn/T/ipykernel_4792/40327047.py:1: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  ipl_df = pd.read_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/ipl_df.csv")


In [8]:
test_df.columns

Index(['match_start_date', 'match_end_date', 'city', 'venue', 'match_type',
       'match_number', 'winner', 'player_of_match', 'team1', 'team2',
       'event_name', 'event_match_number', 'batting_team', 'over', 'batter',
       'bowler', 'non_striker', 'runs_batter', 'runs_extras', 'runs_total',
       'wicket_type', 'player_out', 'fielder'],
      dtype='object')

#### General Information About the data

In [14]:
# Inspect shape and columns
print("Shape:", ipl_df.shape)
print("Columns:\n", ipl_df.columns)

ipl_df.describe()



Shape: (262148, 20)
Columns:
 Index(['match_date', 'city', 'venue', 'match_type', 'match_number', 'winner',
       'player_of_match', 'team1', 'team2', 'batting_team', 'over', 'batter',
       'bowler', 'non_striker', 'runs_batter', 'runs_extras', 'runs_total',
       'wicket_type', 'player_out', 'fielder'],
      dtype='object')


,match_number,over,runs_batter,runs_extras,runs_total
count,246326.000000,262148.000000,262148.000000,262148.000000,262148.000000
mean,31.153065,9.198354,1.266670,0.067855,1.334525
std,18.353378,5.683449,1.641065,0.343409,1.628118
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,4.000000,0.000000,0.000000,0.000000
50%,31.000000,9.000000,1.000000,0.000000,1.000000
75%,46.000000,14.000000,1.000000,0.000000,1.000000
max,72.000000,19.000000,6.000000,7.000000,7.000000


In [15]:
# Check for missing values
print("\nMissing Values:\n", ipl_df.isna().sum())


Missing Values:
 match_date              0
city                12397
venue                   0
match_type              0
match_number        15822
winner               4124
player_of_match       490
team1                   0
team2                   0
batting_team            0
over                    0
batter                  0
bowler                  0
non_striker             0
runs_batter             0
runs_extras             0
runs_total              0
wicket_type        249133
player_out         249133
fielder            252743
dtype: int64


In [16]:
# Check for duplicates
print("\nDuplicates:", ipl_df.duplicated().sum())

# display only te duplicate data from the dataframe
duplicates = ipl_df[ipl_df.duplicated()]
#print("\nDuplicate Rows:\n", duplicates)
duplicates.head(15)


Duplicates: 68567


,match_date,city,venue,match_type,match_number,winner,player_of_match,team1,team2,batting_team,over,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,wicket_type,player_out,fielder
1,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,NaN,NaN,NaN
3,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,NaN,NaN,NaN
4,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,NaN,NaN,NaN
5,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,0,TM Head,K Rabada,Abhishek Sharma,0,0,0,NaN,NaN,NaN
11,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,1,Abhishek Sharma,Arshdeep Singh,TM Head,0,0,0,NaN,NaN,NaN
13,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,2,TM Head,K Rabada,Abhishek Sharma,4,0,4,NaN,NaN,NaN
14,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,2,TM Head,K Rabada,Abhishek Sharma,4,0,4,NaN,NaN,NaN
17,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,2,TM Head,K Rabada,Abhishek Sharma,2,0,2,NaN,NaN,NaN
24,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,3,Nithish Kumar Reddy,Arshdeep Singh,Abhishek Sharma,0,0,0,NaN,NaN,NaN
26,2024-04-09,Mohali,Maharaja Yadavindra Singh International Cricke...,Indian Premier League T20,23.0,Sunrisers Hyderabad,Nithish Kumar Reddy,Sunrisers Hyderabad,Punjab Kings,Sunrisers Hyderabad,4,Abhishek Sharma,SM Curran,Nithish Kumar Reddy,0,0,0,NaN,NaN,NaN


###  The above mentioned values are not the duplicate values because each row is considered as sinlge ball delivery and the data is not duplicate.

In [17]:
# Changing the date format of the match_date column to datetime
ipl_df['match_date'] = pd.to_datetime(ipl_df['match_date'])


In [18]:
def clean_df(df):
    import pandas as pd

    # Show missing values before cleaning
    # print("\n[Before Cleaning] Missing Values:\n", df.isna().sum())

    # Fill specific columns with defaults
    df['wicket_type'] = df['wicket_type'].fillna('No Wicket')
    df['player_out'] = df['player_out'].fillna('None')
    df['fielder'] = df['fielder'].fillna('None')
    df['winner'] = df['winner'].fillna('No result')

    # Fill 'city', 'match_number', 'player_of_match' with 'None' if present
    for col in ['city', 'match_number', 'player_of_match']:
        if col in df.columns:
            df[col] = df[col].fillna('None')

    # Convert appropriate date columns to datetime
    if 'match_date' in df.columns:
        df['match_date'] = pd.to_datetime(df['match_date'])
    if 'match_start_date' in df.columns:
        df['match_start_date'] = pd.to_datetime(df['match_start_date'])
    if 'match_end_date' in df.columns:
        df['match_end_date'] = pd.to_datetime(df['match_end_date'])

    # Convert numerical columns to proper types
    num_cols = ['match_number', 'over', 'runs_batter', 'runs_extras', 'runs_total']
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Show missing values after cleaning
    # print("\n[After Cleaning] Missing Values:\n", df.isna().sum())

    return df

In [19]:
ipl_df = clean_df(ipl_df)
odis_df = clean_df(odis_df)
t20s_df = clean_df(t20s_df)
test_df = clean_df(test_df)



#### 1: Top 10 Run Scorers in IPL


In [11]:

top_batters = (
    ipl_df.groupby('batter')['runs_batter']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

fig = px.bar(
    top_batters,
    x='runs_batter',
    y='batter',
    orientation='h',
    color='runs_batter',
    color_continuous_scale='Magma',
    title='Top 10 Run Scorers in IPL'
)
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

#### 2: Top 10 Wicket-Takers in IPL

In [12]:

top_bowlers = (
    ipl_df[ipl_df['wicket_type'].notnull()]
    .groupby('bowler')
    .size()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='wickets')
)

fig = px.bar(
    top_bowlers,
    x='bowler',
    y='wickets',
    color='wickets',
    title='Top 10 Wicket-Takers in IPL',
    color_continuous_scale='Bluered'
)
fig.show()

#### 3: Dismissal Type Distribution

In [13]:
# Filter out 'No Wicket' and calculate counts
dismissal_counts = (
    ipl_df[ipl_df['wicket_type'] != 'No Wicket']['wicket_type']
    .value_counts()
    .reset_index()
)
dismissal_counts.columns = ['wicket_type', 'count']

# Plot the donut chart
fig = go.Figure(
    data=[go.Pie(
        labels=dismissal_counts['wicket_type'],
        values=dismissal_counts['count'],
        hole=0,
        sort=True
    )]
)
fig.update_layout(title_text="Distribution of Actual Wicket Types (IPL)")
fig.show()


#### 4. Most Successful Teams (Most Wins) 

In [14]:
# Remove 'No result' entries
filtered_df = ipl_df[ipl_df['winner'] != 'No result']
unique_matches = filtered_df.drop_duplicates(subset=['match_date'])
# Count wins by team
team_wins = (
    unique_matches['winner']
    .value_counts()
    .reset_index()
)
team_wins.columns = ['team', 'wins']

# Plot using plotly
fig = px.histogram(
    team_wins,
    x='team',
    y='wins',
    color='wins',
    title='Most Successful Teams in IPL (By Match Wins)'
)
fig.update_xaxes(categoryorder='total descending')
fig.show()

#### 5.Total Runs by City

In [15]:
city_runs = (
    ipl_df.groupby('city')['runs_total']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

fig = px.pie(
    city_runs,
    values='runs_total',
    names='city',
    title='Top 10 Cities by Total Runs Scored (IPL)',
    hole= 0.4 # no hole = full pie
)
fig.show()

#### top 10 bowlers with most dot balls

In [16]:

import plotly.express as px

# Filter dot balls
ipl_dot_balls = ipl_df[ipl_df['runs_batter'] == 0]

# Group by bowler and count
dot_balls_by_bowler = (
    ipl_dot_balls.groupby('bowler')
    .size()
    .reset_index(name='dot_ball_count')
    .sort_values(by='dot_ball_count', ascending=False)
    .head(10)  # top 10 bowlers
)

# Plotly horizontal bar chart
fig = px.bar(
    dot_balls_by_bowler,
    x='dot_ball_count',
    y='bowler',
    orientation='h',
    title='Top 10 Bowlers with Most Dot Balls (IPL)',
    labels={'dot_ball_count': 'Number of Dot Balls', 'bowler': 'Bowler'},
    color='dot_ball_count',
    color_continuous_scale='turbo',
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


### ODI'S TABLE ANALYSIS

#### 1. Identify consistent players who remained not out most often (possibly finishers or resilient top-order batters).


In [22]:
not_out_matches = (
    odis_df[odis_df['player_out'] == 'None']
    .drop_duplicates(subset=['match_date', 'venue', 'batter'])  # One row per match per batter
    .groupby('batter')
    .size()
    .reset_index(name='not_out_matches')
    .sort_values(by='not_out_matches', ascending=False)
    .head(10)
)

fig = px.bar(
    not_out_matches,
    x='batter',
    y='not_out_matches',
    title='Top 10 Batters with Most Not-Out Matches in ODIs',
    color='not_out_matches'
)
fig.show()


#### 2. Overs Analysis: Most Productive Overs in ODIs
      - Insight:To Identify which over number consistently produces the most runs.

In [25]:
avg_runs_per_over = (
    odis_df.groupby('overs')['runs_total']
    .mean()
    .reset_index(name='average_runs')
    .sort_values(by='overs')
)

# Step 2: Use Plotly Bar Chart (non-line version)
fig = px.bar(
    avg_runs_per_over,
    x='overs',
    y='average_runs',
    color='average_runs',
    color_continuous_scale='Viridis',
    title='Average Runs Scored Per Over (ODIs)'
)

fig.update_layout(
    xaxis_title='Over Number',
    yaxis_title='Average Runs',
    bargap=0.2
)

fig.show()

####  3. Team-wise Total Runs Scored in ODIs
- Insight: Compare team-wise batting performance based on total runs scored across all balls.

In [35]:


team_runs = (
    odis_df.groupby('batting_team')['runs_total']
    .sum()
    .reset_index()
    .sort_values(by='runs_total', ascending=False)
    .head(25)
)

fig = px.bar(
    team_runs,
    x='batting_team',
    y='runs_total',
    title='Total Runs Scored by Each Team (ODIs)',
    color='runs_total'
)
fig.update_xaxes(categoryorder='total descending')
fig.show()


#### 3. Bowler Economy Rate Distribution – Histogram (Using Plotly)
- Assuming overs column represents over number (will compute runs conceded per over):
 

In [39]:

# Total runs conceded per bowler and total overs bowled
bowler_stats = (
    odis_df.groupby('bowler')
    .agg({'runs_total': 'sum', 'overs': 'count'})  # count overs (balls bowled)
    .reset_index()
)

# Convert balls to overs (1 over = 6 balls)
bowler_stats['overs_bowled'] = bowler_stats['overs'] / 6
bowler_stats['economy'] = bowler_stats['runs_total'] / bowler_stats['overs_bowled']

fig = px.histogram(
    bowler_stats,
    x='economy',
    nbins=100,
    title='Bowler Economy Rate Distribution (ODIs)'
)
fig.show()

#### 4. Death Over Kings
- Identify batters who score the highest total runs during overs 40–50 in ODIs.


In [ ]:
# Filter for overs 40–50
death_overs_df = odis_df[(odis_df['overs'] >= 40) & (odis_df['overs'] <= 50)]

# Group by batter, calculate average runs in death overs
death_batter_avg = (
    death_overs_df.groupby('batter')['runs_batter']
    .sum()
    .reset_index(name='avg_runs_death_overs')
    .sort_values(by='avg_runs_death_overs', ascending=False)
    .head(10)
)

# Plot as lollipop using Plotly
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=death_batter_avg['batter'],
    y= death_batter_avg['avg_runs_death_overs'],
    mode='markers+lines',
    marker=dict(size=10, color='crimson'),
    line=dict(color='gray', width=2),
    name='Avg Runs'
))

fig.update_layout(
    title='Top 10 Batters in Death Overs (Avg Runs in Overs 40–50)',
    xaxis_title='Batter',
    yaxis_title='Average Runs',
    height=500
)

fig.show()

#### 5. Dot Ball Kings
- Bowlers with highest dot ball percentage (dot = runs_batter == 0)


In [54]:

# We'll go with a radar chart to display dot % across top 5–6 bowlers.


# Total balls bowled per bowler
total_balls = odis_df.groupby('bowler').size().reset_index(name='total_balls')

# Dot balls per bowler
dot_balls = (
    odis_df[odis_df['runs_batter'] == 0]
    .groupby('bowler')
    .size()
    .reset_index(name='dot_balls')
)

# Merge and calculate dot percentage
dot_data = pd.merge(total_balls, dot_balls, on='bowler')
dot_data['dot_percentage'] = (dot_data['dot_balls'] / dot_data['total_balls']) * 100

# Top 6 dot ball bowlers
top_dot = dot_data.sort_values(by='dot_percentage', ascending=False).head(6)

# Radar chart (Plotly)
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=top_dot['dot_percentage'],
    theta=top_dot['bowler'],
    fill='toself',
    name='Dot %',
    marker_color='blue'
))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
    title="🎯 Dot Ball Kings - Highest Dot % Bowlers",
    showlegend=False
)

fig.show()

In [55]:
# Total balls bowled per bowler
total_balls = odis_df.groupby('bowler').size().reset_index(name='total_balls')

# Dot balls per bowler
dot_balls = (
    odis_df[odis_df['runs_batter'] == 0]
    .groupby('bowler')
    .size()
    .reset_index(name='dot_balls')
)

# Merge and calculate dot percentage
dot_data = pd.merge(total_balls, dot_balls, on='bowler')
dot_data['dot_percentage'] = (dot_data['dot_balls'] / dot_data['total_balls']) * 100

# Top 6 bowlers with highest dot %
top_dot = dot_data.sort_values(by='dot_percentage', ascending=False).head(6)

# Doughnut Chart
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(
    labels=top_dot['bowler'],
    values=top_dot['dot_percentage'],
    hole=0.5,
    hoverinfo='label+percent+value',
    textinfo='label+percent'
)])

fig.update_layout(
    title="🎯 Dot Ball Kings - Top Bowlers by Dot Ball %",
    annotations=[dict(text='Dot %', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()


### T20's Data Analysis

In [ ]:

t20s_df = pd.read_csv("/Users/muralidharanv/Documents/GUVI /PROJECTS/Cricket Matchsheet Analysis/Output/t20s_df.csv")
t20_df = clean_df(t20s_df)

#### 1. T20 Team Matchups and Wins Between Teams

In [ ]:
# Step 1: Create a new column for matchup pair (sorted alphabetically to treat both orders the same)
t20s_df['matchup'] = t20s_df.apply(lambda x: ' vs '.join(sorted([x['team1'], x['team2']])), axis=1)

# Step 2: Count total matches and wins per team in each matchup
matchup_summary = t20s_df.groupby(['matchup', 'winner']).size().reset_index(name='match_count')

# Step 3: Get top 25 most frequent matchups
top_matchups = (
    t20s_df['matchup'].value_counts()
    .head(15)
    .reset_index()
    .rename(columns={'total_matches': 'matchup', 'count': 'wins'})
)

# Step 4: Filter matchup_summary to include only those top 25
matchup_summary_top = matchup_summary[matchup_summary['matchup'].isin(top_matchups['matchup'])]

# Step 5: Plot grouped bar chart using Plotly
fig = px.bar(
    matchup_summary_top,
    x='matchup',
    y='match_count',
    color='winner',
    title='Top 15 Most Frequent Team Matchups & Wins',
    labels={'matchup': 'Team Matchup', 'match_count': 'Wins'},
    barmode='group'
)

fig.update_layout(xaxis_tickangle=45, height=600)
fig.show()

#### 2. Death Over Finishers (Overs 16–20)

In [61]:
death_overs_df = t20_df[(t20_df['over'] >= 16) & (t20_df['over'] <= 20)]
death_batters = death_overs_df.groupby('batter')['runs_batter'].sum().reset_index()
death_batters = death_batters.sort_values(by='runs_batter', ascending=False).head(10)

fig2 = go.Figure(data=[go.Pie(
    labels=death_batters['batter'],
    values=death_batters['runs_batter'],
    hole=0.3,
)])
fig2.update_layout(title="Top Finishers in Death Overs (16–20)")
fig2.show()

#### 3. Bowlers With Most Wickets in Powerplay

In [65]:
powerplay_df = t20_df[t20_df['over'] <= 6]
pp_wickets = powerplay_df[powerplay_df['wicket_type'] != 'No Wicket']
bowler_wickets = pp_wickets.groupby('bowler').size().reset_index(name='wickets')
bowler_wickets = bowler_wickets.sort_values(by='wickets', ascending=False).head(10)

fig3 = px.funnel(bowler_wickets, x='wickets', y='bowler', title="Top 10 Powerplay Wicket Takers")
fig3.show()

#### 4. Batters With Highest Strike Rate (min 100 balls)

In [66]:
batter_stats = t20_df.groupby('batter').agg({
    'runs_batter': 'sum',
    'batter': 'count'
}).rename(columns={'batter': 'balls_faced'}).reset_index()
batter_stats['strike_rate'] = (batter_stats['runs_batter'] / batter_stats['balls_faced']) * 100
batter_stats = batter_stats[batter_stats['balls_faced'] >= 100]
top_sr = batter_stats.sort_values(by='strike_rate', ascending=False).head(10)

fig4 = px.scatter(
    top_sr,
    x='runs_batter',
    y='strike_rate',
    size='balls_faced',
    color='batter',
    hover_name='batter',
    title='Top Strike Rates (Min 100 Balls Faced)'
)
fig4.show()

In [70]:

# 5. Economy Kings (Min 50 Balls Bowled)
bowler_stats = t20_df.groupby('bowler').agg({
    'runs_total': 'sum',
    'bowler': 'count'
}).rename(columns={'bowler': 'balls_bowled'}).reset_index()
bowler_stats = bowler_stats[bowler_stats['balls_bowled'] >= 50]
bowler_stats['economy'] = (bowler_stats['runs_total'] / bowler_stats['balls_bowled']) * 6
economy_kings = bowler_stats.sort_values(by='economy').head(10)

fig5 = px.histogram(
    economy_kings,
    x='bowler',
    y='economy',
    color='economy',
    title="Top 10 Bowlers with Best Economy (Min 50 Balls)"
)
fig5.update_traces(marker_line_width=1)
fig5.show()

#### 6. Powerplay Blasters: Avg Runs in First 6 Overs (T20)

In [ ]:


# Filter for Powerplay overs (0 to 5)
powerplay_df = t20_df[t20_df['over'] < 6]

# Total runs by team in powerplay across matches
powerplay_runs = powerplay_df.groupby(['venue', 'batting_team'])['runs_total'].sum().reset_index()

# Average runs in powerplay
avg_powerplay = powerplay_runs.groupby('batting_team')['runs_total'].mean().reset_index().head(25)
avg_powerplay.columns = ['team', 'avg_powerplay_runs']

# Plot the bar chart
fig = px.bar(
    avg_powerplay.sort_values(by='avg_powerplay_runs', ascending=False),
    x='team',
    y='avg_powerplay_runs',
    title='Powerplay Blasters: Avg Runs in First 6 Overs (T20)',
    color='avg_powerplay_runs',
    color_continuous_scale='YlOrRd',
)

fig.update_layout(xaxis_title='Team', yaxis_title='Avg Powerplay Runs', height=500)
fig.show()


### TEST Mathces Analysis

#### 1. Longest Individual Innings (By Balls Faced)
- Insight: Identify batters who have faced the most balls in a single match — a measure of grit and concentration.

In [ ]:
long_innings = (
    test_df[test_df['runs_batter'].notna()]
    .groupby(['venue', 'batter'])['runs_batter']
    .count()
    .reset_index(name='balls_faced')
    .sort_values(by='balls_faced', ascending=False)
    .head(10)
)

fig1 = px.funnel(
    long_innings,
    x='balls_faced',
    y='batter',
    title='Longest Individual Innings by Balls Faced (Top 10)'
)
fig1.show()

#### 2. Bowling Consistency: Most Maiden Overs
- Insight: Bowlers who consistently applied pressure by conceding no runs in an over.

In [101]:
# Filter for dot balls
dot_balls = test_df[test_df['runs_total'] == 0]

# Count dot balls per over
maiden_overs = (
    dot_balls.groupby(['venue', 'bowler', 'over']).size()
    .reset_index(name='dot_count')
    .query('dot_count == 6')  # Maiden = 6 dot balls
)

maiden_count = maiden_overs.groupby('bowler').size().reset_index(name='maiden_overs')
maiden_count = maiden_count.sort_values(by='maiden_overs', ascending=False).head(10)

fig2 = px.pie(
    maiden_count,
    names='bowler',
    values='maiden_overs',
    title='Top 10 Bowlers with Most Maiden Overs',
    hole=0.3
)
fig2.show()

#### 4. Multi-Day Match Duration Trends
- Insight: Duration of matches by number of days played.

In [103]:



test_df['match_start_date'] = pd.to_datetime(test_df['match_start_date'])
test_df['match_end_date'] = pd.to_datetime(test_df['match_end_date'])
test_df['match_duration_days'] = (test_df['match_end_date'] - test_df['match_start_date']).dt.days + 1

match_durations = (
    test_df[['venue', 'match_duration_days']]
    .drop_duplicates()
    .sort_values(by='match_duration_days', ascending=False)
)

fig4 = px.box(
    match_durations,
    y='match_duration_days',
    title='Distribution of Test Match Durations (in Days)'
)
fig4.show()

#### Overs with 2+ Wickets
> Insight: Overs where teams lost momentum — losing 2 or more wickets in an over.


In [106]:

wicket_overs = (
    test_df[test_df['wicket_type'] != 'No Wicket']
    .groupby(['venue', 'over'])['player_out']
    .count()
    .reset_index(name='wickets_in_over')
    .query('wickets_in_over >= 2')
)

fig5 = px.histogram(
    wicket_overs,
    x='wickets_in_over',
    nbins=5,
    title='Frequency of Overs with Multiple Wickets'
)
fig5.show()

In [109]:
## who is the batter who has faced most overs and scored most runs in test matches
batter_stats = (
    test_df.groupby('batter')
    .agg({'runs_batter': 'sum', 'batter': 'count'})
    .rename(columns={'batter': 'balls_faced'})
    .reset_index()
    .sort_values(by=['runs_batter', 'balls_faced'], ascending=False)
    .head(10)
)
fig6 = px.scatter(
    batter_stats,
    x='runs_batter',
    y='balls_faced',
    size='balls_faced',
    color='batter',
    hover_name='batter',
    title='Top Test Batters by Runs and Balls Faced'
)
fig6.show()
